The aim of this notebook is to use the inbuilt supervised and unsupervised learning capabilities inside gee to train some basic models for predicting peat presence and peat depth. This will form some of the basic building blocks of the more advanced modelling.

The first section of the notebook involves obtaining the relevant data to make the models (e.g. Sentinel-2 imagery limited to the Fens and cloud-cover dealt with).

I will try to build the following basic models:
1. A random forest regression model - model peat depth using the labelled dataset that we have. We will use only Sentinel 2 imagery as the predictor. 
2. Multi-predictor RF model - build the same model but this time add an additional second layer to the model e.g. lidar data
3. Linear regression model - see if we can build models that use linear regression to compare performance (generally we would expect RF to do better). 
4. Unsupervised model - we should use the inbuilt capabilities to build a basic presence/absence model using Sentinel 2 data only. In the future, to limit runtime, we may need a presence/absence model to limit the prediction area.
5. Completion - on completion of these four models, we should start looking at more advanced modelling methods such as AI/ML which will require exploration of the TensorFlow examples given. 

## Setup

In [1]:
# necessary imports 
import ee
import geemap

# if error with geopandas then try running:
# pip install --upgrade --force-reinstall shapely

In [2]:
# data and file paths
path_depths = "/Users/hamishcampbell/Documents/Cambridge/PhD/1st Year/Code/depth-mapping/Data/depth_samples.shp" 
path_fenOutline = "/Users/hamishcampbell/Documents/Cambridge/PhD/1st Year/Code/depth-mapping/Data/NCA_46_The_Fens.shp"

In [3]:
# initialise Google Earth Engine

# NOTE: first run only
#ee.Authenticate()

ee.Initialize()

## Load data into GEE

In [4]:
# setup the base map for adding layers to
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [6]:
# load the Fens outline shape file into GEE
fenOutline = ee.FeatureCollection(geemap.shp_to_ee(path_fenOutline))

In [7]:
# get all surface reflectance Sentinel-2 imagery 
sen = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")

# keep only the data that meets some basic criteria
sen_filtered = (
    sen.filterBounds(fenOutline)                    # spatially limited to tiles overlapping Fens
    .filterDate('2022-01-01','2023-01-01')          # temporally limited to 2022 imagery
    .sort('CLOUD_COVERAGE_ASSESSMENT', False)       # sort tiles by cloud cover
    .select(['B[2-4]','MSK_CLDPRB']))               # only RGB and pixel probability of cloud presence  
    

In [8]:
# Write a function that alters the pixel validity mask of a tile according to if cloud is present or not

# if MSK_CLDPRB less than 10 => low chance of cloud => let mask remain valid at that pixel
# if medium-high chance of cloud (>10) => update mask to say pixel is invalid (mask = 0)
def check_cloud_absence(tile): 
    return tile.updateMask(tile.select('MSK_CLDPRB').lte(10))

In [9]:
# obtain a (mostly) cloud-free mosaic for each quarter 
quarterly_mosaics = []
filter_start = ee.Date('2022-01-01')

for quarter in range(0,4):
    
    # keep only tiles from the quarter being studied
    quarterly_sen = sen_filtered.filterDate(filter_start, filter_start.advance(3,'month'))
    
    # Update the pixel validity mask to include condition on cloud absence
    quarterly_sen = quarterly_sen.map(check_cloud_absence)
    
    # stitch the most cloud-free tiles together and replace remaining cloud pixels with those from next tile with presence
    # NOTE: by sorting on cloud presence we keep complete image as temporally contiguous as possible
    quarterly_mosaic = quarterly_sen.mosaic()
    
    # store the result
    quarterly_mosaics.append(quarterly_mosaic)
    
    # repeat for next quarter
    filter_start = filter_start.advance(3,'month')
    

In [10]:
# select the quarter of the year to use for modelling (1/2/3/4)
quarter = 2

In [15]:
# Add the result to the map to check result
visParams = {
  'bands': ['B4', 'B3', 'B2'],
  'max': 3000,
  'min': 0,
}
Map.addLayer(quarterly_mosaics[quarter-1], visParams, 'Sentinel Mosaic')

In [16]:
# plot the outline in black with a transparent fill
style = {
    'color': 'red',
    'fillColor': '00000000'
}
Map.centerObject(fenOutline, 8)
Map.addLayer(fenOutline.style(**style), {}, 'Fen Outline')

In [17]:
# load the depth measurements shape file into GEE
depths = geemap.shp_to_ee(path_depths)

# plot the points at which ground-truth measurements were taken on the map
style = {
    'color': 'blue',
    'pointSize': 2,
    'pointShape': 'circle'
}
Map.addLayer(depths.style(**style), {}, 'Depth Measurements')
Map

Map(bottom=21738.0, center=[52.70465287691509, 0.062270855934294995], controls=(WidgetControl(options=['positi…